In [ ]:
from transformers import ViTForImageClassification, ViTImageProcessor
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# print cuda device information
print(torch.cuda.get_device_name(0))

In [ ]:

# Load Pre-Trained ViT Model
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=100,  # Adjust for CIFAR-100
    id2label={i: str(i) for i in range(100)},
    label2id={str(i): i for i in range(100)},
    ignore_mismatched_sizes=True  # Ignore size mismatch for the classification head
)
model.to(device)


In [ ]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')

In [ ]:
# Data Preprocessing
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std),
])

# Load CIFAR-100 Dataset
train_dataset = datasets.CIFAR100(root="./data", train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR100(root="./data", train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
def train_and_evaluate_model(model, train_loader, test_loader, optimizer, criterion, device, num_epochs):
    train_acc = []
    test_acc = []
    for epoch in range(num_epochs):
        # Initialize a new progress bar for each epoch
        progress_bar = tqdm(total=len(train_loader) + len(test_loader), desc=f"Epoch {epoch + 1}/{num_epochs} - Train Acc: 0.000 - Val Acc: 0.000", unit="batch")
        
        # Initialize variables for training phase
        train_correct = 0
        train_total = 0
        
        # Training Phase
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(pixel_values=images)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()
            
            _, predicted = outputs.logits.max(1)
            train_correct += predicted.eq(labels).sum().item()
            train_total += labels.size(0)
            
            # Update training accuracy
            train_accuracy = train_correct / train_total
            progress_bar.set_description(f"Epoch {epoch + 1}/{num_epochs} - Train Acc: {train_accuracy:.4f}")
            progress_bar.update(1)  # Update progress bar for each training batch
        
        train_acc.append(train_accuracy)
        
        # Initialize variables for validation phase
        val_correct = 0
        val_total = 0
        
        # Validation Phase
        model.eval()
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(pixel_values=images)
                loss = criterion(outputs.logits, labels)
                
                _, predicted = outputs.logits.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)
                
                # Update validation accuracy
                val_accuracy = val_correct / val_total
                progress_bar.set_description(f"Epoch {epoch + 1}/{num_epochs} - Train Acc: {train_accuracy:.4f} - Val Acc: {val_accuracy:.4f}")
                progress_bar.update(1)  # Update progress bar for each validation batch
                
        test_acc.append(val_accuracy)

        # Reset the progress bar description for the next epoch
        progress_bar.set_description(f"Epoch {epoch + 1}/{num_epochs} - Train Acc: {train_accuracy:.4f} - Val Acc: {val_accuracy:.4f}")
        progress_bar.close()
        
    return train_acc, test_acc

# Call the function
num_epochs = 5
# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01)
train_acc, test_acc = train_and_evaluate_model(model, train_loader, test_loader, optimizer, criterion, device, num_epochs)

# Save the Fine-Tuned Model
model.save_pretrained("./vit-finetuned-cifar100")

In [ ]:
# print table of train and test accuracy
print("Epoch\tTrain Acc\tTest Acc")
for i in range(num_epochs):
    print(f"{i + 1}\t{train_acc[i]:.4f}\t{test_acc[i]:.4f}")


In [ ]:
# Plot train and test accuracy
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(range(1, num_epochs + 1), train_acc, label="Train Accuracy", marker="o")
plt.plot(range(1, num_epochs + 1), test_acc, label="Test Accuracy", marker="o")
plt.title("Training Progress")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend()
plt.xticks(range(1, num_epochs + 1))
plt.show()

In [ ]:
from huggingface_hub import PyTorchModelHubMixin
model.push_to_hub("Swapnil949/vit-finetuned-cifar100", use_temp_dir=True)

In [ ]:
import time
correct = 0
total = 0

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

model.eval()
model.to(device)

# Start timing
start_time = time.time()

# Add a tqdm progress bar
for images, labels in tqdm(test_loader, desc="Processing batches", unit="img"):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = outputs.logits.max(1)
    correct += predicted.eq(labels).sum().item()
    total += labels.size(0)

# End timing
end_time = time.time()

# Calculate and print total elapsed time
elapsed_time = end_time - start_time
print(f"Accuracy: {correct / total:.4f}")
print(f"Total inference time: {elapsed_time:.2f} seconds")
print(f"Average time per image: {elapsed_time / len(test_loader):.4f} seconds")
